# Project 03: Gibbs Sampling (Random Algorithm)

In [ ]:
import bamnostic as bs
import seqlogo
from pprint import pprint
import numpy as np
import pandas as pd

#import function for building sequence motif & idenfitying seqs matching to motif
from project03 import GibbsMotifFinder, run_parallel
import motif_ops
from utils import utils
from sequence_database import sequence_box

from MACS3.IO.Parser import BAMPEParser
from MACS3.Signal.Region import Regions
from pyfaidx import Fasta

print("Great, everything is up to date")

ModuleNotFoundError: No module named 'MACS3.Signal.CutoffAnalysis'

# Our code
Check notes in each cell to see if it's what you're trying to do

In [12]:
#create the seqs array, just need to do 1 time and it will take a while. For Vic it took ~3-5 minutes.
bam_path = "data/SRR9090854.subsampled_5pct.bam"

bam = bs.AlignmentFile(bam_path)


In [11]:
standard_chroms = [str(i) for i in range(1, 23)] + ['X', 'Y']

regions_to_be_excluded = Regions()
for entry in bam.header['SQ']:
    chrom = str(entry['SN'])
    chrom_length = entry['LN']
    if chrom not in standard_chroms:
        regions_to_be_excluded.add_loc(chrom.encode(), 0, chrom_length)

In [ ]:
genome = Fasta('data/GCF_000009045.1_ASM904v1_genomic.fna')


In [35]:
parser = BAMPEParser(bam_path)
petrack = parser.build_petrack()
petrack.finalize()
pileup_track = petrack.pileup_bdg()

(pileup_sum, pileup_length, pileup_max, pileup_min, pileup_mean, pileup_std) = pileup_track.summary()
print(f"{pileup_sum=}\n{pileup_length=}\n{pileup_max=}\n{pileup_min=}\n{pileup_mean=}\n{pileup_std=}")

global_bg_track = pileup_track.set_single_value(pileup_mean) # this will return a new track with all values set as `pileup_mean`
score_track = pileup_track.make_ScoreTrackII_for_macs(global_bg_track, depth1=100, depth2=100) # Note, depth1 and 2 are the same so there is no need for scaling the values


INFO  @ 12 Feb 2026 17:33:24: [518 MB]  1000000 fragments parsed 
INFO  @ 12 Feb 2026 17:33:27: [523 MB] 1543642 fragments have been read. 


pileup_sum=14850177024.0
pileup_length=3100499056
pileup_max=2374.0
pileup_min=0.0
pileup_mean=4.789608955383301
pileup_std=6.075918197631836


In [36]:
score_track.change_score_method(ord('q'))
cutoff_analysis = score_track.cutoff_analysis(min_score=0.5, max_score=50)
print(cutoff_analysis)

AttributeError: 'MACS3.Signal.ScoreTrack.ScoreTrackII' object has no attribute 'cutoff_analysis'

In [ ]:
from io import StringIO
import matplotlib.pyplot as plt

cutoff_analysis_df = pd.read_csv(StringIO(cutoff_analysis), sep="\t")
fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharex=True)

axes[0].plot(cutoff_analysis_df["score"], cutoff_analysis_df["npeaks"])
axes[0].set_title("npeaks")
axes[0].set_xlabel("score")
axes[0].set_ylabel("value")

axes[1].plot(cutoff_analysis_df["score"], cutoff_analysis_df["lpeaks"])
axes[1].set_title("lpeaks")
axes[1].set_xlabel("score")

axes[2].plot(cutoff_analysis_df["score"], cutoff_analysis_df["avelpeak"])
axes[2].set_title("avelpeak")
axes[2].set_xlabel("score")

plt.tight_layout()
plt.show()